In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
# from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
from pyCRMtools.pycaps import arps_read
from pyCRMtools.pycaps import pycaps_fields
from pyCRMtools.pycaps import calvars_radar as pycaps_radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
%matplotlib inline

In [ ]:
PIPS_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/TriPIPS/2019'
parsivel_combined_filename = 'parsivel_combined_TriPIPS_092719_TriPIPS_10s.nc'
parsivel_combined_path = os.path.join(PIPS_dir, parsivel_combined_filename)
parsivel_combined_ds = xr.open_dataset(parsivel_combined_path)

In [ ]:
print(parsivel_combined_ds)

In [ ]:
radar_at_PIPS_da = parsivel_combined_ds['xtrra_at_PIPS']
print(radar_at_PIPS_da)
#print(radar_at_PIPS_da.coords['fields'].values)
fields = radar_at_PIPS_da.coords['fields_xtrra'].values
field_name = next((fname for fname in fields if fname in radar.REF_aliases))
print(field_name)

In [ ]:
from natsort import natsorted
radar_name = 'XTRRA'
radar_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/XTRRA/2019/09/27/CFRadial'
radar_paths = glob.glob(radar_dir + '/xtrra_20190927-[1-2][9,0]*.nc')
radar_paths = natsorted(radar_paths)
print(radar_paths)

In [ ]:
sweeptime_start = radar._getsweeptime(radar_paths[0])
sweeptime_stop = radar._getsweeptime(radar_paths[-1])
print(sweeptime_start, sweeptime_stop)

radar_paths2 = []
sweeptimes2 = []
el_req = 2.4
for radar_path in radar_paths:
    elevation = radar._getelev(radar_path)
    sweeptime = radar._getsweeptime(radar_path)
    if np.abs(elevation - el_req) < 0.1:
        print(elevation)
        print(os.path.basename(radar_path))
        radar_paths2.append(radar_path)
        sweeptimes2.append(sweeptime)
        

In [ ]:
starttime = '20190927190015'
stoptime = '20190927205801'

radar_dict = radar.read_sweeps(radar_paths2, starttime, stoptime)
print(radar_dict['radarsweeplist'])

In [ ]:
# GPS_stats = conv_df['GPS_status']
# GPS_lats = conv_df['GPS_lat']
# GPS_lons = conv_df['GPS_lon']
# GPS_alts = conv_df['GPS_alt']
# PIPS_lat, PIPS_lon, PIPS_alt = pipsio.get_PIPS_loc(GPS_stats, GPS_lats, GPS_lons, GPS_alts)
PIPS_loc_str = parsivel_combined_ds.location
PIPS_lat, PIPS_lon, PIPS_alt = list(map(np.float, PIPS_loc_str.strip('()').split(',')))
PIPS_rad_loc = radar.get_PIPS_loc_relative_to_radar((PIPS_lat, PIPS_lon), radar_dict['radarsweeplist'][0])
print(PIPS_rad_loc)

In [ ]:
import pyart

for radar_obj, sweeptime in zip(radar_dict['radarsweeplist'], sweeptimes2):
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    distance = np.sqrt((PIPS_rad_loc[0] - xrad)**2. + (PIPS_rad_loc[1] - yrad)**2.)
    theta_index, range_index = np.unravel_index(distance.argmin(), distance.shape)

    print(theta_index, range_index)

    display = pyart.graph.RadarDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    # plot reflectivity
    ax = fig.add_subplot(111)
    print("Time: ", sweeptime)
    print("Elevation: ", radar_obj.elevation['data'][-1])
    display.plot('corrected_reflectivity', 0, title='Reflectivity',
                 vmin=0, vmax=60, colorbar_label='', ax=ax)
    # plot ZDR
    
#     display.plot('differential_reflectivity', 0, title='ZDR',
#                  vmin=0, vmax=6, colorbar_label='', ax=ax)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-250, 250), ylim=(-250, 250))
    display.set_limits(xlim=(-20, 0), ylim=(-5, 15), ax=ax)
    ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'k*')
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

In [ ]:
radar_obj = radar_dict['radarsweeplist'][0]
print(radar_obj.info())

In [ ]:
print(radar_obj.fields['reflectivity'])